In [ ]:
"""
Physics-Informed Convolutional model to predict
the diagonal Reynolds stress tensor terms, which
can later be used to calculate Turbulent Pressure.

The example data is a 1D mapped version of the 3D CCSN
12 solar mass simulation from Burrows et al, 2020.

Note: only select checkpoints are provided for this example,
      while the full mapped dataset is available through 
      documentation

The algorithm is best suited to be trained on CPU/s,
given the limited data and the shallow network employed.

Note: due to a custom loss implementation, Catalyst is NOT used,
hence logging is handled manually for this example!

-pikarpov
"""

%matplotlib inline

import os
import sys
import numpy as np
from scipy import interpolate
import scipy.signal as sig

sys.path.append("../../")

from sapsan.lib.backends import MLflowBackend, FakeBackend
from sapsan.lib.data import HDF5Dataset
from sapsan.lib.estimator.pimlturb1d.pimlturb1d_estimator import PIMLTurb1D, PIMLTurb1DConfig
from sapsan.lib import Train, Evaluate
from sapsan.utils.plot import log_plot, line_plot

In [ ]:
# --- Experiment tracking backend ---
# MLflow - the server will be launched automatically
# in case it won't, type in cmd: mlflow ui --port=5000
# to use it, set mlflow = True

mlflow = False
experiment_name = "PIMLTurb 1D experiment"

if mlflow: 
    tracking_backend = MLflowBackend(experiment_name, host="localhost", port=5000)
else: tracking_backend = FakeBackend()

In [ ]:
# --- Data setup ---
# In the intereset of loading and training multiple timesteps
# one can specify which checkpoints to use and where
# they appear in the path via syntax: {checkpoint:format}
# 
# Next, you need to specify which features to load; let's assume 
#         path = "{feature}.h5"
# 
#  1. If in different files, then specify features directly;
#     The default HDF5 label will be the last label in the file
#     Ex: features = ['velocity', 'denisty', 'pressure']
#  2. If in the same, then duplicate the name in features
#     and specify which labels to pull
#     Ex: features = ["data", "data", "data"]
#         feature_labels = ['velocity', 'density', 'pressure']

smass = 's12.0'
base  = 'data/1dccsn/%s.swbj15.horo.3d/'%smass
path  = base+'dump_{checkpoint:05d}.h5'

# rho, Pgas, Vsound, T, entropy
features_label = ['rho','eos0','eos1','eos2','eos3']
target_label   = ['Pturb']

# Target will be Pgas/Pturb which will need to be calculated separately
target         = ['dummy']

# Checkpoints for training
checkpoints = [30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 
               130, 140, 150, 160, 170, 180, 190, 200, 210] 
               
# Checkpoints for validation
checkpoints_valid = [35, 65, 95, 125, 155, 185]

# Checkpoints to load
checkpoints_load = checkpoints + checkpoints_valid

# Dimensionality of your data in format (D,H,W)
# It is 1D in this case, with 678 grid points in radius
INPUT_SIZE = [678]

# Interpolation size to equite the region between the proto neutron star
# and the stalled shock across all checkpoints
mlin_grid_size = 200  

In [ ]:
# Data pre-processing functions to:
#   1. find proto neutron star position 
#   2. find shock position
#   3. interpolate the region between them
#   4. scale the data

def shock_position(x, v, pturb_pgas):    
    shock_r = 0    
    shock_i = np.argmin(v)
    
    # find the local minima right before the shock to exclude 
    # the diffused shock (otherwise can contaminate Pturb/Pgas prediction)
    minima  = np.array(sig.argrelmin(pturb_pgas[:shock_i]))[0]
    
    # a few extra grid points to be safe against the diffused shock
    buffer = 2
    
    for j in range(len(minima)-1,-1,-1):        
        shock_i = minima[j]-buffer
        if pturb_pgas[shock_i]<1: break
    shock_r = x[shock_i]
    
    return shock_i
        
def pns_radius(rho, rho_threshold=1e12):  
    # find when density is reaching the threshold for the proto neutron star density  
    for i in range(np.size(rho)-1,-1,-1):
        if rho[i] >= rho_threshold:
            pns_x = rho[i]
            pns_ind = i
            break
    return pns_ind

def process_var(var, v, new_r, pns_i, shock_i):
    f = interpolate.interp1d(np.arange(np.size(v))[pns_i:shock_i+1], var[pns_i:shock_i+1])
    return f(new_r)

def interpolate_data(x, y, velocity, mlin_grid_size):
    
    mlin_x = np.zeros((x.shape[0], x.shape[1], mlin_grid_size))
    mlin_y = np.zeros((y.shape[0], y.shape[1], mlin_grid_size))
    
    for i in range(x.shape[0]):
        v          = velocity[i]
        pturb_pgas = y[i,0]
        
        shock_i = shock_position(np.arange(np.size(v)), v, pturb_pgas)
        pns_i   = pns_radius(x[i,0])
        
        new_r = np.linspace(pns_i,shock_i,mlin_grid_size)                           
        
        for j in range(x.shape[1]): 
            mlin_x[i,j] = process_var(x[i,j], v, new_r, pns_i,shock_i)
            
        mlin_y[i,0] = process_var(y[i,0], v, new_r, pns_i,shock_i)  
            
        print(f'Checkpoint ind {i} | Convective region from/to: {pns_i}, {shock_i} | Total size: {shock_i-pns_i}')

    return mlin_x, mlin_y, pns_i,shock_i

def scale_data(mlin_x, mlin_y, units, features_label):
    # scale input fatures and target based on provided units
    
    print('----------')
    for i in range(mlin_x.shape[1]):    
        mlin_x[:,i] *= units[features_label[i]]
        print(f'{features_label[i]:10s} min: {np.amin(mlin_x[:,i]):.3e}, max: {np.amax(mlin_x[:,i]):.3e}')   
        
    mlin_y[:,0] *= units[target_label[0]]
    print(f'{target_label[0]}/Pgas min: {np.amin(mlin_y[:,0]):.3e}, max: {np.amax(mlin_y[:,0]):.3e}')   
    print('----------')
    
    return mlin_x, mlin_y

In [ ]:
# Load the radial grid to find proto neutron star and shock positions
grid =  HDF5Dataset(path           = base+'grid.h5',
                    features_label = ['Z'],
                    checkpoints    = [0],
                    input_size     = INPUT_SIZE)
r = grid.load_numpy().flatten()[:-1]

# Load the Time metadata
data_loader = HDF5Dataset(path           = path,                         
                          features_label = ['Time'],                          
                          checkpoints    = checkpoints_load,
                          input_size     = INPUT_SIZE,                          
                          shuffle        = False,
                          train_fraction = len(checkpoints))

# in miliseconds
time = data_loader.load_numpy().flatten()

# Load velocity profiles to find the shock position
data_loader = HDF5Dataset(path           = path,                         
                          features_label = ['u1'],                          
                          checkpoints    = checkpoints_load,
                          input_size     = INPUT_SIZE,                          
                          shuffle        = False,
                          train_fraction = len(checkpoints))

velocity = data_loader.load_numpy()

# Load the training and validation data
data_loader = HDF5Dataset(path           = path,
                          target         = target,
                          features_label = features_label,
                          target_label   = target_label,
                          checkpoints    = checkpoints_load,
                          input_size     = INPUT_SIZE,
                          shuffle        = False,
                          train_fraction = len(checkpoints))

x, y = data_loader.load_numpy()

# Calculate the target Pturb/Pgas
for i in range(x.shape[1]):
    if 'eos0' in features_label[i]: y[:,0] = np.divide(y[:,0],x[:,i])

# Find the proto neutron star and the shock position, and interpolate
# the convective in-between region to mlin_grid_size for each checkpoint
mlin_x, mlin_y, pns_i, shock_i = interpolate_data(x, y, velocity, mlin_grid_size)

# Scaling coefficients to equilibrate all maximums closer to 1e0
# (leaving dependency on Pgas to be greater than others)
units = {'u1'   : 1,
         'mach2': 1,
         'rho'  : 2e-12,
         'eos0' : 2e-31,
         'eos1' : 1e-8,
         'eos2' : 1,
         'eos3' : 1e-1,
         target_label[0]:1/3}

mlin_x, mlin_y = scale_data(mlin_x, mlin_y, units, features_label)

loaders = data_loader.convert_to_torch([mlin_x, mlin_y])

In [ ]:
# Machine Learning model to use

# Configuration of the model parameters:
#    n_epochs  = number of epochs (need ~1000 for sensible results using PIMLTurb1D)
#    patience  = number of epochs to run beyond convergence (not used for SmoothL1_KSLoss)
#    min_delta = loss based convergence cut-off (not used for SmoothL1_KSLoss)
#    lr        = learning rate
#    min_lr    = minimum learning rate
#    device    = device to train the model on: ['cpu', 'cuda', 'cuda:0', ...]
#    activ     = activation function
#    loss      = loss function
#    ks_stop   = value of the KS (Kolmogorov-Smirnov stat) to stop training,
#                checks both 'train' and 'valid' KS values 
#                Note: NOT the total loss value, but only the KS component!
#    ks_frac   = contribution fraction of KS to the total loss. l1_frac = 1-ks_frac
#    l1_scale  = amount to scale the initial L1 loss by, so that it is
#                reduced first, with KS loss dominating later in the training
#    ks_scale  = amount to scale the initial KS loss by (should always be = 1)
#    l1_beta   = beta for the Smooth L1 (cutoff point for smooth portion)
#    sigma     = sigma for the Gaussian kernel for the filtering/smoothing layer

estimator = PIMLTurb1D(
        config   = PIMLTurb1DConfig(n_epochs   = 20, 
                                    patience   = 10, 
                                    min_delta  = 1e15, 
                                    lr         = 1e-4, 
                                    min_lr     = 1e-4*1e-5, 
                                    device     = 'cpu', 
                                    loader_key = 'train'),
        loaders  = loaders,
        activ    = "Tanhshrink", 
        loss     = "SmoothL1_KSLoss",
        ks_stop  = 0.13,
        l1_scale = 1e3,
        ks_scale = 1,
        l1_beta  = 1,
        sigma    = 2)

In [ ]:
#Set the experiment
training_experiment = Train(model           = estimator,
                            backend         = tracking_backend,
                            data_parameters = data_loader)

#Train the model
estimator = training_experiment.run()
if mlflow: 
    tracking_backend.log_parameter("ks_stop",  ks_stop)
    tracking_backend.log_parameter("ks_scale", ks_scale)
    tracking_backend.log_parameter("l1_scale", l1_scale)
    tracking_backend.log_parameter("units",    units) 

In [ ]:
# plotting training results
# Note: not done automatically since Catalyst is NOT used
log_path       = 'train_log.txt'
valid_log_path = 'valid_log.txt'
delimiter      = '\t'

log = log_plot(show_log       = True,
               log_path       = log_path, 
               valid_log_path = valid_log_path, 
               delimiter      = delimiter)

log_name = 'runtime_log.html'
log.write_html(log_name)
tracking_backend.log_artifact(log_name)

data       = np.genfromtxt(log_path, delimiter=delimiter, 
                            skip_header=1, dtype=np.float32)
data_valid = np.genfromtxt(valid_log_path, delimiter=delimiter, 
                            skip_header=1, dtype=np.float32)
    
tracking_backend.log_metric('train - loss', data[-1,1])
tracking_backend.log_metric('valid - loss', data_valid[-1,1])
tracking_backend.log_metric('train - final epoch', data[-1,0])  

#--------------------------------------------------
# plotting individual evolutions of L1 and KS losses

losses = [[f"mean(L1_loss)", f"mean(L1_valid)"],
          [f"mean(KS_loss)", f"mean(KS_valid)"]]

valid_log_path = 'valid_l1ks_log.txt'
log_path       = 'train_l1ks_log.txt'

for idx, (train_name, valid_name) in enumerate(losses):
    if   'L1' in train_name: mark='L1'; idx = 2
    elif 'KS' in train_name: mark='KS'; idx = 1
    
    log = log_plot(show_log       = True, 
                   valid_log_path = valid_log_path,
                   log_path       = log_path,
                   delimiter      = delimiter,
                   train_name     = train_name, 
                   valid_name     = valid_name,
                   train_column   = idx, 
                   valid_column   = idx, 
                   epoch_column   = None)
    
    log_name = f"{train_name}.html"
    log.write_html(log_name)    
    tracking_backend.log_artifact(log_name) 
    
    data       = np.genfromtxt(log_path, delimiter=delimiter, 
                                skip_header=1, dtype=np.float32)
    data_valid = np.genfromtxt(valid_log_path, delimiter=delimiter, 
                            skip_header=1, dtype=np.float32)
        
    tracking_backend.log_metric(f'train - {mark} loss', data[-1,idx])
    tracking_backend.log_metric(f'valid - {mark} loss', data_valid[-1,idx])

In [ ]:
target_checkpoint = [205]

# Load velocity profiles to find the shock position
data_loader = HDF5Dataset(path           = path,                         
                          features_label = ['u1'],                          
                          checkpoints    = target_checkpoint,
                          input_size     = INPUT_SIZE,                          
                          shuffle        = False,
                          train_fraction = len(checkpoints)-len(checkpoints_valid))

velocity = data_loader.load_numpy()

#Load the test data
data_loader_eval = HDF5Dataset(path           = path,
                               target         = target,
                               features_label = features_label,
                               target_label   = target_label,
                               checkpoints    = target_checkpoint,
                               input_size     = INPUT_SIZE)

x, y = data_loader_eval.load_numpy()

# Calculate the target Pturb/Pgas
for i in range(x.shape[1]):
    if 'eos0' in features_label[i]: y[:,0] = np.divide(y[:,0],x[:,i])
        
# Find the proto neutron star and the shock position, and interpolate
# the convective in-between region to mlin_grid_size for each checkpoint
mlin_x, mlin_y, pns_i, shock_i = interpolate_data(x, y, velocity, mlin_grid_size)

mlin_x, mlin_y = scale_data(mlin_x, mlin_y, units, features_label)

loaders = data_loader_eval.convert_to_torch([mlin_x, mlin_y])

#Set the test experiment
estimator.loaders = loaders
data_loader_eval.input_size=[mlin_grid_size]
data_loader_eval.batch_size=[mlin_grid_size]

evaluation_experiment = Evaluate(model           = estimator,
                                 backend         = tracking_backend,
                                 data_parameters = data_loader_eval)

# Let's make an extra plot from the evaluation
# evaluation returns a dict, cubes = {'pred_cube':np.ndarray, 'target_cube':np.ndarray}
cubes = evaluation_experiment.run() 

ax = line_plot([[np.arange(mlin_grid_size),cubes['target'][0,0]],
                [np.arange(mlin_grid_size),cubes['predict'][0,0]]],
               label     = ['target', 'predict'],
               plot_type ='semilogy', 
               figsize   = (10,6))

ax.set_xlabel('index')
ax.set_ylabel(r'$P_{turb}/P_{gas}$')
ax.set_title(f'checkpoint={target_checkpoint[0]}')
ax.set_ylim(1e-6,1e1)
ax.legend(loc=2)
ax.get_figure().tight_layout()

# If mlflow is active, then we want to open the last evaluation ID
# and add this new plot to it, instead of creating a new experiment entry
if mlflow:
    logy_name = 'spatial_plot_logy.png'
    ax.get_figure().savefig(logy_name)
    
    run_id = evaluation_experiment.run_id
    tracking_backend.resume(run_id = run_id)
    tracking_backend.log_artifact(logy_name)           
    tracking_backend.end()       